In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder
  .appName("StructuredNetworkWordCount")
  .config("spark.master", "local")
  .getOrCreate()

import spark.implicits._

spark = org.apache.spark.sql.SparkSession@29c7dbbb


org.apache.spark.sql.SparkSession@29c7dbbb

In [2]:

// Create DataFrame representing the stream of input lines from connection to host:port
// val lines = spark.readStream
//   .format("socket")
//   .option("host", host)
//   .option("port", port)
//   .load()
val path = "./text"
val lines = spark.readStream
  .format("text")
  .option("maxFilesPerTrigger", 1)
  .option("fileNameOnly", true)
  .load(path)

// Split the lines into words
val words = lines.as[String].flatMap(_.split(" "))

// Generate running word count
val wordCounts = words.groupBy("value").count()

// Start running the query that prints the running counts to the console
val query = wordCounts.writeStream
  .outputMode("complete")
  .format("console")
  .start()

query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-----+
|               value|count|
+--------------------+-----+
|              online|    1|
|              graphs|    1|
|          ["Building|    1|
|          ["Parallel|    1|
|              thread|    1|
|       documentation|    3|
|            command,|    2|
|         abbreviated|    1|
|            overview|    1|
|                rich|    1|
|                 set|    2|
|         -DskipTests|    1|
|                name|    1|
|page](http://spar...|    1|
|        ["Specifying|    1|
|              stream|    1|
|                 not|    1|
|                run:|    1|
|            programs|    2|
|     ./dev/run-tests|    1|
+--------------------+-----+
only showing top 20 rows



Name: java.lang.InterruptedException
Message: null
StackTrace:   at java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:998)
  at java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1304)
  at java.util.concurrent.CountDownLatch.await(CountDownLatch.java:231)
  at org.apache.spark.sql.execution.streaming.StreamExecution.awaitTermination(StreamExecution.scala:467)
  at org.apache.spark.sql.execution.streaming.StreamingQueryWrapper.awaitTermination(StreamingQueryWrapper.scala:53)

In [14]:
val df = spark.readStream
  .format("text")
  .option("maxFilesPerTrigger", 1)
  .load("words.txt")

df = [value: string]


lastException: Throwable = null


[value: string]

⊕ [spark-structured-streaming-book/spark-sql-streaming-MemoryStream.adoc at master · jaceklaskowski/spark-structured-streaming-book](https://github.com/jaceklaskowski/spark-structured-streaming-book/blob/master/spark-sql-streaming-MemoryStream.adoc)


In [6]:
implicit val ctx = spark.sqlContext

import org.apache.spark.sql.execution.streaming.MemoryStream
import org.apache.spark.sql.functions._

// It uses two implicits: Encoder[Int] and SQLContext
val intsIn = MemoryStream[Int]

val ints = intsIn.toDF
  .withColumn("t", current_timestamp())
  .withWatermark("t", "5 minutes")
  .groupBy(window($"t", "5 minutes") as "window")
  .agg(count("*") as "total")

import org.apache.spark.sql.streaming.{OutputMode, Trigger}
import scala.concurrent.duration._
val totalsOver5mins = ints.
  writeStream.
  format("memory").
  queryName("totalsOver5mins").
  outputMode(OutputMode.Append).
  trigger(Trigger.ProcessingTime(10.seconds)).
  start

ctx = org.apache.spark.sql.SQLContext@1ecf86ae
intsIn = MemoryStream[value#33]
ints = [window: struct<start: timestamp, end: timestamp>, total: bigint]
totalsOver5mins = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@30d392c1


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@30d392c1

In [7]:
val zeroOffset = intsIn.addData(0, 1, 2)

zeroOffset = 0


0

In [8]:
totalsOver5mins.processAllAvailable()
spark.table("totalsOver5mins").show

+------+-----+
|window|total|
+------+-----+
+------+-----+



In [14]:
// intsOut.show
totalsOver5mins.stop()